In [4]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from NetLiquidity import nlq_clean as nq
from NetLiquidity import nlq_chart as plotit

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


AttributeError: module 'matplotlib' has no attribute 'backends'

In [ ]:
inputs = pd.read_excel(r"/home/purpy-furcat/Documents/Code/Bootleg_Macro/Liquidity/NetLiquidity/NetLiquidity_InputParams.xlsx", index_col=0)
inputs

In [ ]:
start_date = inputs.loc["Start date", "Additional FRED Data"]
start_date

In [ ]:
nlq_datas = nq.NetLiquidity()

In [ ]:
nq_results = nlq_datas.calculate_all()

In [ ]:
nq_results.keys()

In [ ]:
netliq = nq_results["nlq_daily_treasury"]

In [ ]:
from MacroBackend import charting_plotly as chp
from MacroBackend import Pull_Data as pul

In [ ]:
btc = pul.dataset()
btc.get_data("tv", "BTCUSD", start_date="2014-09-17", exchange_code="INDEX")

In [ ]:
bitty = btc.data
bitty.plot()

In [ ]:
figdog = plotit.plot_nlq_dual_axis(nlq_data=netliq, asset_data={"Bitcoin": bitty.loc["01-01-2021":]}, title="Net Liquidity vs Assets")

In [ ]:
# Example 2: Chart with custom equally-spaced ticks
import Liquidity.NetLiquidity.nlq_chart as plot


fig2, ax_l2, ax_r2 = plot.plot_nlq_dual_axis(
    nlq_data=netliq,
    asset_data={"Bitcoin": bitty.loc["01-01-2021":]}, 
    title="Net Liquidity vs BTC",
    ma_period=50,
    left_scale='log',
    right_scale='log',
    custom_ticks_left=8,
    custom_ticks_right=10,
    left_tick_prefix='$',
    right_tick_prefix='$',
    left_ticks_int=True,
    right_ticks_int=True
)
fig2.show()

In [ ]:
    # Example 3: Using equal_spaced_ticks function for plotly
tick_config = plot.equal_spaced_ticks(
    num_ticks=10,
    data=netliq,
    scale='log',
    lab_prefix='$',
    lab_suffix='B',
    return_format='plotly'
)
print("Plotly tick config:", tick_config)

In [ ]:
fig = chp.dual_axis_basic_plot(primary_data=netliq, secondary_data=bitty.loc["01-01-2021":], template="ggplot2",
                               log_primary=True, log_secondary=True, title="Net Liquidity vs Bitcoin Price",
                               primary_yaxis_title="Net liquidity (USD Billions)", secondary_yaxis_title=" Price (USD)",     custom_ticks_primary=8,
    custom_ticks_secondary=10, height = 600,
    primary_tick_prefix='$',
    secondary_tick_prefix='$',
    primary_ticks_int=True,
    secondary_ticks_int=True)
fig.show()


In [ ]:
from MacroBackend import stats

In [ ]:
pair = stats.Pair_stats(netliq, bitty.loc["01-01-2021":])

In [ ]:
ma_shifted = netliq.rolling(window=50).mean().shift(-30)

In [ ]:
scat = pair.bm_scatterMatrix(yoy=True, show=True)

In [ ]:
# Example 2: Chart with custom equally-spaced ticks
import Liquidity.NetLiquidity.nlq_chart as plot
import matplotlib.pyplot as plt

fig2, ax_l2, ax_r2 = plot.plot_nlq_dual_axis(
    nlq_data=netliq,
    asset_data={"Bitcoin": bitty.loc["01-01-2021":]}, 
    title="Net Liquidity vs BTC",
    ma_period=50,
    left_scale='log',
    right_scale='log',
    custom_ticks_left=8,
    custom_ticks_right=10,
    left_tick_prefix='$',
    right_tick_prefix='$',
    left_ticks_int=True,
    right_ticks_int=True
)
ax_l2.plot(ma_shifted.index, ma_shifted.values, label='50-Day MA Shifted', color='orange')
fig2.show()

In [ ]:
pair2 = stats.Pair_stats(ma_shifted, bitty.loc["01-01-2021":ma_shifted.index[-1]])

In [ ]:
pair2.plot_corrs()
plt.show()

In [ ]:
btc_yoy = bitty.loc["01-01-2021":ma_shifted.index[-1]].dropna()
mashift_yoy = ma_shifted.pct_change(365).dropna()
pair3 = stats.Pair_stats(btc_yoy, mashift_yoy)

In [ ]:
pair3.plot_series()
plt.show()